In [ ]:
"""
git é uma merda! srsrs
"""

In [1]:
import os
import json
import tweepy

In [2]:
# Leitura das chaves
with open("keys.txt","r") as text:
    text = text.read()
    keys = json.loads(text)

print(keys.keys())

dict_keys(['API_KEY', 'API_KEY_SECRET', 'ACCESS_TOKEN', 'ACCESS_TOKEN_SECRET', 'BEARER_TOKEN'])


In [3]:
def conect_api():
    auth = tweepy.OAuth1UserHandler(
      keys['API_KEY'], 
      keys['API_KEY_SECRET'], 
      keys['ACCESS_TOKEN'], 
      keys['ACCESS_TOKEN_SECRET'])
    
    return tweepy.API(auth)

In [4]:
api = conect_api()

#### Verificação das credenciais

In [5]:
"""
my_credentials = api.verify_credentials()
my_credentials
""";

### testando a API com uma busca simples de tweets publicados com a hashtag pantanal

In [6]:
"""
query = "burguer king"
locale = ""
language = "pt"
""";

In [7]:
"""
tweets = api.search_tweets(q = query, lang = language, tweet_mode = "extended")

for tweet in tweets:
    try:
        print(tweet.retweeted_status.full_text)
        print("=====")
    except AttributeError:
        print(tweet.full_text)
        print("=====")
""";

## capturando tweets com o uso de cursores

In [8]:
#help(tweepy.Cursor)

In [9]:
#help(tweepy.OAuth1UserHandler)

#### LISTAGEM DE RESUATRANTES

In [27]:
restaurantes = {"mcdonalds","méqui","burger king", "girafas", "subway",
                "madero", "starbucks", "pizza hut","pizzahut", "dominos",
                "fogo de chao", "habibs"}

#### SENTIMENTOS

In [45]:
sentimentos_positivos = {"😍", "🥰", "😋","🤩","😀","😇"}

In [46]:
sentimentos_negativos = {"🤮","🤢","🤬","😡","😠","💩","😣"}

### MINERACAO DE DADOS

In [47]:
publicacoes = []

In [48]:
for restaurante in restaurantes:
    query = restaurante

    try:
        for status in tweepy.Cursor(api.search_tweets, q = query, lang="pt",
                                    count=200).items(10000):
            publicacoes.append(status)
    except:
        print("limite de requisicao estourado!")

limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!


In [14]:
# for restaurante in restaurantes:
#     query = restaurante
    
#     try:
#         for status in tweepy.Cursor(api.search_tweets, q = query, lang="pt",
#                                     count=200,exclude_replies=True, include_rts=False).items(10000):
#             publicacoes.append(status)
#     except:
#         print("limite de requisicao estourado!")

limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!
limite de requisicao estourado!


In [49]:
len(publicacoes)

17597

In [59]:
"""
for pub in publicacoes:
    print(pub.text)
    print(pub.created_at)
    print("-------")
""";

### passando os principais dados para um DataFrame

In [54]:
def cria_data_frame():
    from pandas import DataFrame
    id_autor, texto, data_publicacao = [], [], []

    for pub in publicacoes:
        id_autor.append(pub.id_str)
        texto.append(pub.text)
        data_publicacao.append(pub.created_at)
    
    main_data = DataFrame({"id_autor":id_autor, "texto":texto , "data_publicacao":data_publicacao })
    
    return main_data


In [55]:
df = cria_data_frame()
df.head()

,id_autor,texto,data_publicacao
0,1531725294154960896,"Era só um bk, subway ou sushizinho agr",2022-05-31 19:52:22+00:00
1,1531725243471040513,Mudando de pau pra casete o Vontade de comer u...,2022-05-31 19:52:10+00:00
2,1531725045541789696,vontade de subway,2022-05-31 19:51:23+00:00
3,1531724677365776385,Subway o comida criolla?,2022-05-31 19:49:55+00:00
4,1531723997133553672,a cada decepção os pontos do Subway surf aumenta,2022-05-31 19:47:13+00:00


### salvando os dados

In [56]:
# separando as colunas pot \t para não dar problemar ao carregar os dados em um novo df
df.to_csv("raw-data/coleta-1.csv", sep="\t", index=False)

In [57]:
df.data_publicacao.min(), df.data_publicacao.max()

(Timestamp('2022-05-23 08:14:36+0000', tz='UTC'),
 Timestamp('2022-05-31 19:56:38+0000', tz='UTC'))